In [72]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os


pd.set_option('display.max_columns', None)

## Accidents
### Read in Accidents DF

In [74]:
df = gpd.read_file('/Users/Dario/Documents/GitHub/GIS_KARTO/data/accidents.json')

keep_cols = ['AccidentType_de', 'AccidentSeverityCategory_de', 'AccidentInvolvingPedestrian', 'AccidentInvolvingBicycle', 'RoadType_de', 'AccidentLocation_CHLV95_E', 'AccidentLocation_CHLV95_N', 'CantonCode', 'AccidentYear', 'AccidentMonth', 'geometry' ]

In [75]:
gdf = df.drop(columns=[col for col in df.columns if col not in keep_cols])

In [76]:
# Typechange of DF

gdf['AccidentInvolvingBicycle'] = gdf['AccidentInvolvingBicycle'].str.lower().map({'true': True, 'false': False})
gdf['AccidentInvolvingPedestrian'] = gdf['AccidentInvolvingPedestrian'].str.lower().map({'true': True, 'false': False})
gdf.AccidentYear = gdf.AccidentYear.astype(int)
gdf.AccidentMonth = gdf.AccidentMonth.astype(int)
gdf.AccidentLocation_CHLV95_E = gdf.AccidentLocation_CHLV95_E.astype(int)
gdf.AccidentLocation_CHLV95_N = gdf.AccidentLocation_CHLV95_N.astype(int)



In [77]:
# Filtering DF
#gdf = gdf[gdf['AccidentYear'] <= 2018]
gdf = gdf[(gdf['AccidentMonth'] <= 9) & (gdf['AccidentYear'] <= 2018)]
gdf = gdf[gdf['AccidentInvolvingBicycle']]

In [16]:
#gdf.to_file('../../export/accidents/acc.geojson', index=False)

## Master Table

In [150]:
# Read MasterTable 

master_table = pd.read_csv('../../data.csv', sep=',')
master_table['geometry'] = master_table['geometry'].apply(wkt.loads)
master = gpd.GeoDataFrame(master_table, geometry='geometry')

## Compare Accidents with Master Table

In [151]:
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon

In [152]:
gdf = gdf.set_crs(gdf.crs)

geometry = gpd.points_from_xy(gdf['AccidentLocation_CHLV95_E'], gdf['AccidentLocation_CHLV95_N'], crs='epsg:2056')

gdf.geometry = geometry

In [153]:
# Load the GeoDataFrames
polygons_gdf = master_table #resulate df
points_gdf = gdf #acc df

In [154]:
# Subset for faster computation
small_poly = gpd.GeoDataFrame(polygons_gdf.head(20), crs='epsg:2056')
small_point = gpd.GeoDataFrame(points_gdf.head(20), crs='epsg:2056')

In [155]:
# Perform the spatial join
#joined_gdf = sjoin(small_point, small_poly, op='within')

# Count the number of points per polygon and create a new column in the polygons GeoDataFrame
#acc_count = joined_gdf.groupby('index_right').size().reset_index(name='Acc_Count')
#small_poly = small_poly.merge(acc_count, on='index_right', how='left')


In [156]:
small_poly['Acc_Count'] = 0

In [157]:
small_poly

,GMDNR,GMDNAME_x,Stimmberechtigte,Abgegebene Stimmen,Beteiligung in %,Gültige Stimmzettel,Ja,Nein,Ja in %,GMDNAME_y,BZNR,KTNR,E_CNTR,N_CNTR,geometry,Acc_Count
0,1,Aeugst am Albis,1402,664,47.36,663,481,182,72.55,Aeugst am Albis,101,1,2679300,1235700,"POLYGON ((2678219.000 1235219.000, 2678439.000...",0
1,2,Affoltern am Albis,7186,2880,40.08,2850,2079,771,72.95,Affoltern am Albis,101,1,2676800,1236800,"POLYGON ((2678219.000 1235219.000, 2677940.000...",0
2,3,Bonstetten,3656,1701,46.53,1691,1251,440,73.98,Bonstetten,101,1,2677800,1241000,"POLYGON ((2675803.000 1241039.000, 2675748.000...",0
3,4,Hausen am Albis,2505,1145,45.71,1132,802,330,70.85,Hausen am Albis,101,1,2682900,1233100,"POLYGON ((2686324.000 1230388.000, 2686024.000...",0
4,5,Hedingen,2515,1183,47.04,1178,876,302,74.36,Hedingen,101,1,2676400,1239000,"POLYGON ((2675803.000 1241039.000, 2676538.000...",0
5,6,Kappel am Albis,712,281,39.47,280,187,93,66.79,Kappel am Albis,101,1,2682400,1231300,"POLYGON ((2680285.000 1229552.000, 2679649.000...",0
6,7,Knonau,1482,591,39.88,584,445,139,76.2,Knonau,101,1,2677500,1230900,"POLYGON ((2679323.000 1230263.000, 2679649.000...",0
7,8,Maschwanden,461,197,42.73,196,133,63,67.86,Maschwanden,101,1,2674800,1232000,"POLYGON ((2673859.000 1233032.000, 2673861.000...",0
8,9,Mettmenstetten,3378,1740,51.51,1725,1230,495,71.3,Mettmenstetten,101,1,2677600,1233000,"POLYGON ((2678219.000 1235219.000, 2678385.000...",0
9,10,Obfelden,3472,1298,37.38,1287,902,385,70.09,Obfelden,101,1,2675100,1235300,"POLYGON ((2673859.000 1233032.000, 2673590.000...",0


In [148]:
for idx, point in small_point.geometry.items():
    # loop through the small_poly GeoDataFrame with index and polygon
    for poly_idx, poly in small_poly.geometry.items():
        # check if the point is within the polygon
        if point.within(poly):
            # increment the count of points for the polygon
            print(point, poly_idx)
            small_poly.at[poly_idx, 'Acc_Count'] += 1
        else:
            pass

In [149]:
small_poly

,GMDNR,GMDNAME_x,Stimmberechtigte,Abgegebene Stimmen,Beteiligung in %,Gültige Stimmzettel,Ja,Nein,Ja in %,GMDNAME_y,BZNR,KTNR,geometry,Acc_Count
0,1,Aeugst am Albis,1402,664,47.36,663,481,182,72.55,Aeugst am Albis,101,1,"POLYGON ((2678219.000 1235219.000, 2678439.000...",0
1,2,Affoltern am Albis,7186,2880,40.08,2850,2079,771,72.95,Affoltern am Albis,101,1,"POLYGON ((2678219.000 1235219.000, 2677940.000...",0
2,3,Bonstetten,3656,1701,46.53,1691,1251,440,73.98,Bonstetten,101,1,"POLYGON ((2675803.000 1241039.000, 2675748.000...",0
3,4,Hausen am Albis,2505,1145,45.71,1132,802,330,70.85,Hausen am Albis,101,1,"POLYGON ((2686324.000 1230388.000, 2686024.000...",0
4,5,Hedingen,2515,1183,47.04,1178,876,302,74.36,Hedingen,101,1,"POLYGON ((2675803.000 1241039.000, 2676538.000...",0
5,6,Kappel am Albis,712,281,39.47,280,187,93,66.79,Kappel am Albis,101,1,"POLYGON ((2680285.000 1229552.000, 2679649.000...",0
6,7,Knonau,1482,591,39.88,584,445,139,76.2,Knonau,101,1,"POLYGON ((2679323.000 1230263.000, 2679649.000...",0
7,8,Maschwanden,461,197,42.73,196,133,63,67.86,Maschwanden,101,1,"POLYGON ((2673859.000 1233032.000, 2673861.000...",0
8,9,Mettmenstetten,3378,1740,51.51,1725,1230,495,71.3,Mettmenstetten,101,1,"POLYGON ((2678219.000 1235219.000, 2678385.000...",0
9,10,Obfelden,3472,1298,37.38,1287,902,385,70.09,Obfelden,101,1,"POLYGON ((2673859.000 1233032.000, 2673590.000...",0


In [117]:
joined = gpd.sjoin(small_poly, small_point, op='contains')

# group the resulting GeoDataFrame by the polygons and count the number of points
counts = joined.groupby('GMDNR').size().reset_index(name='Acc_Count')

# merge the counts back to the small_poly GeoDataFrame
small_poly = small_poly.merge(counts, on='GMDNR', how='left')

/Users/Dario/opt/anaconda3/envs/GTA22/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3472: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
